In [ ]:
import pandas as pd
import numpy as np
import pickle
import logging
import math

from app.data_engineering.data_access import read_db
from app.data_engineering.feature_engineering import FeatureEngineering

import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# To plot pretty figures
%matplotlib inline
import matplotlib.pyplot as plt
plt.rcParams['axes.labelsize'] = 14
plt.rcParams['xtick.labelsize'] = 12
plt.rcParams['ytick.labelsize'] = 12

pd.set_option('display.max_rows', 1000)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

In [ ]:
datasets = read_db()

In [ ]:
datasets.keys()

In [ ]:
df_batch1 = datasets['batch1']
df_batch2 = datasets['batch2']
df_test_vols = datasets['test']['vols']

In [ ]:
FEATURES = df_test_vols.columns.tolist()

In [ ]:
LABELS = list(set(df_batch1['vols'].columns.tolist()) - set(df_test_vols.columns.tolist()))

In [ ]:
label = "RETARD A L'ARRIVEE"

In [ ]:
COLUMNS = FEATURES + [label]

## df_batch1 & df_batch2

In [ ]:
df_batch1.keys()

In [ ]:
#df_batch1['vols'].loc[:, 'DATE'] = pd.to_datetime(df_batch1['vols']['DATE'])
#df_batch2['vols'].loc[:, 'DATE'] = pd.to_datetime(df_batch2['vols']['DATE'])

In [ ]:
#df_vol1 = df_batch1['vols']
#df_vol2 = df_batch2['vols']

In [ ]:
#df_vol1.to_csv('../data/batch_1_vol.csv', index=False)
#df_vol2.to_csv('../data/batch_2_vol.csv', index=False)

In [ ]:
df_vol1 = pd.read_csv('../data/batch_1_vol.csv')
df_vol2 = pd.read_csv('../data/batch_2_vol.csv')

In [ ]:
df_vols = pd.concat([df_vol1, df_vol2])

### Vols

In [ ]:
# remove already handle line
df_vols = df_vols.dropna(
    subset=["TEMPS DE DEPLACEMENT A TERRE A L'ATTERRISSAGE", 
            "TEMPS DE DEPLACEMENT A TERRE AU DECOLLAGE",
            "RETARD A L'ARRIVEE"]
)

In [ ]:
df_vols

In [ ]:
#    def fill_temps_de_deplacement_annulation(self, df):
#        df.loc[df['ANNULATION']==1, [
#            "TEMPS DE DEPLACEMENT A TERRE AU DECOLLAGE", 
#            "TEMPS DE DEPLACEMENT A TERRE A L'ATTERRISSAGE", "RETARD A L'ARRIVEE"]] = 999
#        df.loc[(df["DETOURNEMENT"]==1) & (df["TEMPS DE DEPLACEMENT A TERRE A L'ATTERRISSAGE"].isna()), [
#            "TEMPS DE DEPLACEMENT A TERRE A L'ATTERRISSAGE", "RETARD A L'ARRIVEE"]] = 999

class DataCleaning:
    def __init__(self, features_columns, label):
        self.features_columns = features_columns
        self.label = label
        self.cie_by_avion = None

    def remove_unused_columns(self, df):
        if 'NIVEAU DE SECURITE' in df.columns:
            df = df.drop(columns=['NIVEAU DE SECURITE'])
        return df

    def cleaning(self, df):
        df = df.dropna(subset=self.features_columns)
        if self.label in df.columns:
            df = df.dropna(subset=[self.label])
        return df
    
    def fill_na(self, df):
        if self.cie_by_avion is None:
            self.cie_by_avion = df[['CODE AVION', 'COMPAGNIE AERIENNE']].dropna().groupby(
                by=['CODE AVION']).first().to_dict()['COMPAGNIE AERIENNE']
        
        df.loc[df['COMPAGNIE AERIENNE'].isna(), 'COMPAGNIE AERIENNE'] = df.loc[
            df['COMPAGNIE AERIENNE'].isna(), 'CODE AVION'].apply(
                lambda x: self.cie_by_avion.get(x, "UKN")
            )
        return df
    
    def transform(self, df):
        df = df.copy()
        df = self.fill_na(df)
        #df = self.cleaning(df)
        #df = self.remove_unused_columns(df)
        #df.loc[:, 'DATE'] = pd.to_datetime(df['DATE'])
        return df


In [ ]:
data_cleaning = DataCleaning(features_columns=FEATURES, label=label)
df_vols_cleaned = data_cleaning.transform(df_vols)

In [ ]:
display(df_vols_cleaned.head(30))
display(df_vols_cleaned.shape)

In [ ]:
df_vols_cleaned[COLUMNS].info(show_counts=True) 

In [ ]:
df_vols_cleaned[COLUMNS].nunique()

### Dataviz for vols batch 1 and 2

In [ ]:
df_vols_eng = df_vols_cleaned.copy()

In [ ]:
df_vols_eng['TEMPS DE DEPLACEMENT A TERRE AU DECOLLAGE'] = df_vols_eng['TEMPS DE DEPLACEMENT A TERRE AU DECOLLAGE'].apply(lambda x: math.sqrt(x))
df_vols_eng["TEMPS DE DEPLACEMENT A TERRE A L'ATTERRISSAGE"] = df_vols_eng["TEMPS DE DEPLACEMENT A TERRE A L'ATTERRISSAGE"].apply(lambda x: math.sqrt(x))
df_vols_eng = df_vols_eng[df_vols_eng['NOMBRE DE PASSAGERS']<1000] 
df_vols_eng = df_vols_eng[df_vols_eng["RETARD A L'ARRIVEE"]<150] #ou bien 
#df_vols_eng.loc[df_vols_eng["RETARD A L'ARRIVEE"]>500, "RETARD A L'ARRIVEE"] = 501



In [ ]:
for column in ['AEROPORT DEPART',
                'AEROPORT ARRIVEE',
                'DEPART PROGRAMME',
                'ARRIVEE PROGRAMMEE',
                'TEMPS PROGRAMME',
                'TEMPS DE DEPLACEMENT A TERRE AU DECOLLAGE',
                "TEMPS DE DEPLACEMENT A TERRE A L'ATTERRISSAGE",
                'DISTANCE',
                'COMPAGNIE AERIENNE',
                'NOMBRE DE PASSAGERS',
                'DATE',
                "RETARD A L'ARRIVEE"]:
    #fig = px.histogram(df_vols_cleaned, x=column, nbins=min(200, df_vols_cleaned[column].nunique()))
    #fig.show()
    print(column)
    df_vols_eng[column].hist(figsize=(15,8), bins=100)
    plt.show();

In [ ]:
df_vols_cleaned[df_vols_cleaned['TEMPS DE DEPLACEMENT A TERRE AU DECOLLAGE']>150].head()[COLUMNS]

In [ ]:
df_vols_cleaned["TEMPS A TERRE"] = df_vols_cleaned['TEMPS DE DEPLACEMENT A TERRE AU DECOLLAGE'] +\
    df_vols_cleaned["TEMPS DE DEPLACEMENT A TERRE A L'ATTERRISSAGE"]
df_vols_cleaned_smaller = df_vols_cleaned.sample(frac=0.1)

for column in ['AEROPORT DEPART',
                'AEROPORT ARRIVEE',
                'DEPART PROGRAMME',
                'ARRIVEE PROGRAMMEE',
                'TEMPS PROGRAMME',
                'TEMPS DE DEPLACEMENT A TERRE AU DECOLLAGE',
                "TEMPS DE DEPLACEMENT A TERRE A L'ATTERRISSAGE",
                "TEMPS A TERRE",
                'DISTANCE',
                'COMPAGNIE AERIENNE',
                'NOMBRE DE PASSAGERS',
                'DATE',]:
    plt.close()
    fig = plt.figure(figsize=(12,8))
    ax = plt.axes()

    plt.rcParams.update({'font.size': 15})

    plt.scatter(df_vols_cleaned_smaller[column], df_vols_cleaned_smaller["RETARD A L'ARRIVEE"],
             c='#FF0000')

    plt.title(f"RETARD A L'ARRIVEE par {column}", size=20, color="#000070")

    # Titres des axes
    ax = ax.set(xlabel=column, ylabel="RETARD A L'ARRIVEE")
    plt.show()

### Aeroports

In [ ]:
df_aeroports = pd.concat([df_batch1['aeroports'], df_batch2['aeroports']])
df_aeroports.drop_duplicates(inplace = True)

duplicate_airport = df_aeroports.loc[
    df_aeroports.duplicated(subset=['CODE IATA'], keep=False),
    ['CODE IATA', 'PRIX RETARD PREMIERE 20 MINUTES', 'PRIS RETARD POUR CHAQUE MINUTE APRES 10 MINUTES']].groupby(
    by=['CODE IATA']).mean().to_dict(orient='index')

df_aeroports.drop_duplicates(inplace = True, subset=['CODE IATA'])
df_aeroports[df_aeroports['CODE IATA']=='JNB'] # => duplicate on JNB if no subset

In [ ]:
for code_iata in duplicate_airport.keys():
    df_aeroports.loc[
        df_aeroports['CODE IATA']==code_iata,
        ['PRIX RETARD PREMIERE 20 MINUTES']
    ] = df_aeroports.loc[
        df_aeroports['CODE IATA']==code_iata,
        ['PRIX RETARD PREMIERE 20 MINUTES']
    ].apply(lambda x: duplicate_airport[code_iata]['PRIX RETARD PREMIERE 20 MINUTES'], axis=1)
    
    df_aeroports.loc[
        df_aeroports['CODE IATA']==code_iata,
        ['PRIS RETARD POUR CHAQUE MINUTE APRES 10 MINUTES']
    ] = df_aeroports.loc[
        df_aeroports['CODE IATA']==code_iata,
        ['PRIS RETARD POUR CHAQUE MINUTE APRES 10 MINUTES']
    ].apply(lambda x: duplicate_airport[code_iata]['PRIS RETARD POUR CHAQUE MINUTE APRES 10 MINUTES'], axis=1)

In [ ]:
df_aeroports[df_aeroports['CODE IATA']=='JNB'] # => duplicate on JNB if no subset

In [ ]:
display(df_aeroports.head(20))
display(df_aeroports.shape)

In [ ]:
df_aeroports.info(show_counts=True) 

In [ ]:
df_aeroports.nunique()

In [ ]:
df_aeroports.describe()

In [ ]:
df_aeroports['LONGITUDE'] = df_aeroports['LONGITUDE'].astype('float')
df_aeroports['LATITUDE'] = df_aeroports['LATITUDE'].astype('float')

In [ ]:
df_aeroports['LONGITUDE TRONQUEE'] = df_aeroports['LONGITUDE'].apply(round)
df_aeroports['LATITUDE TRONQUEE'] = df_aeroports['LATITUDE'].apply(round)

#### Retour sur la df VOLS

In [ ]:
airport_dict = df_aeroports.set_index("CODE IATA").to_dict(orient='index')

In [ ]:
df_vols_eng.loc[:, 'PAYS DEPART'] = df_vols_eng['AEROPORT DEPART'].apply(lambda x: airport_dict.get(x)['PAYS'])
df_vols_eng.loc[:, 'PAYS ARRIVEE'] = df_vols_eng['AEROPORT ARRIVEE'].apply(lambda x: airport_dict.get(x)['PAYS'])

df_vols_eng.loc[:, 'HAUTEUR DEPART'] = df_vols_eng['AEROPORT DEPART'].apply(lambda x: airport_dict.get(x)['HAUTEUR'])
df_vols_eng.loc[:, 'HAUTEUR ARRIVEE'] = df_vols_eng['AEROPORT ARRIVEE'].apply(lambda x: airport_dict.get(x)['HAUTEUR'])
df_vols_eng.loc[:, 'LONGITUDE ARRIVEE'] = df_vols_eng['AEROPORT ARRIVEE'].apply(lambda x: airport_dict.get(x)['LONGITUDE TRONQUEE'])
df_vols_eng.loc[:, 'LATITUDE ARRIVEE'] = df_vols_eng['AEROPORT ARRIVEE'].apply(lambda x: airport_dict.get(x)['LATITUDE TRONQUEE'])

df_vols_eng.loc[:, 'PRIX RETARD PREMIERE 20 MINUTES'] = df_vols_eng['AEROPORT ARRIVEE'].apply(
    lambda x: airport_dict.get(x)['PRIX RETARD PREMIERE 20 MINUTES'])
df_vols_eng.loc[:, 'PRIS RETARD POUR CHAQUE MINUTE APRES 10 MINUTES'] = df_vols_eng['AEROPORT ARRIVEE'].apply(
    lambda x: airport_dict.get(x)['PRIS RETARD POUR CHAQUE MINUTE APRES 10 MINUTES'])


In [ ]:
df_vols_eng

In [ ]:
for column in ['PAYS DEPART',
               'PAYS ARRIVEE',
               'HAUTEUR DEPART',
               'HAUTEUR ARRIVEE',
               'LONGITUDE ARRIVEE',
               'LATITUDE ARRIVEE',
               'PRIX RETARD PREMIERE 20 MINUTES',
               'PRIS RETARD POUR CHAQUE MINUTE APRES 10 MINUTES'
              ]:
    print(column)
    df_vols_eng[column].hist(figsize=(15,8), bins=100)
    plt.show();

In [ ]:
df_vols_eng_smaller = df_vols_eng.sample(frac=0.1)

for column in ['PAYS DEPART',
               'PAYS ARRIVEE',
               'HAUTEUR DEPART',
               'HAUTEUR ARRIVEE',
               'LONGITUDE ARRIVEE',
               'LATITUDE ARRIVEE',
               'PRIX RETARD PREMIERE 20 MINUTES',
               'PRIS RETARD POUR CHAQUE MINUTE APRES 10 MINUTES']:
    plt.close()
    fig = plt.figure(figsize=(12,8))
    ax = plt.axes()

    plt.rcParams.update({'font.size': 15})

    plt.scatter(df_vols_eng_smaller[column], df_vols_eng_smaller["RETARD A L'ARRIVEE"],
             c='#FF0000')

    plt.title(f"RETARD A L'ARRIVEE par {column}", size=20, color="#000070")

    # Titres des axes
    ax = ax.set(xlabel=column, ylabel="RETARD A L'ARRIVEE")
    plt.show()

### Compagnies

In [ ]:
df_compagnies1 = pd.concat([df_batch1['compagnies'], df_batch2['compagnies']]).drop_duplicates()
display(df_compagnies1)
display(df_compagnies1.shape)

### Prix du fuel

## Test

In [ ]:
df_test_vols

In [ ]:
df_test_vols_cleaned = data_cleaning.transform(df_test_vols)
df_test_vols_cleaned.loc[:, 'DATE'] = pd.to_datetime(df_test_vols_cleaned['DATE'])

# DATAVIZ